In [ ]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora, models, similarities
import numpy as np
import logging
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

lemmatizer = Mystem()
stemmer = SnowballStemmer('russian', ignore_stopwords=True)
stop = stopwords.words('russian')
TRY_NAME = "ALL_STEM_LEM"
stop.extend (["\n", " "])
files = ['xaa','xab','xac','xad','xae','xaf','xag','xah','xai','xaj','xak','xal']

In [ ]:
def file_reader ():
    for f in files:
        with open ("../temp/" + f + "_result") as fin:
            for l in fin:
                res = l.decode ("utf-8").lower ().strip ().split("\t")
                idx = res[0]
                title = ""
                body = ""
                if len (res) > 1:
                    title = res[1]
                if len (res) > 2:
                    body = res[2]
                yield idx, title, body

In [ ]:
dct = Dictionary(prune_at=None)

def apply_to_str (string):
    no_stops = filter (lambda x: x not in stop, lemmatizer.lemmatize (string))
    return map(lambda x: stemmer.stem(x), no_stops)

for idx, title, body in tqdm.tqdm (file_reader (), total=582167):
    dct.add_documents ([title.split (), body.split ()], prune_at=None)

dct.save ("../result/{}/dict.dct".format (TRY_NAME))

In [ ]:
sample[sample['QueryId'] == 0]

In [ ]:
sample = pd.read_csv('../data/sample.csv', sep=',')

In [ ]:
q2docs = []
for i in range(6311):
    q2docs.append(list(sample[sample['QueryId'] == i]['DocumentId']))

In [ ]:
sample.size

In [ ]:
sample[sample["DocumentId"] == sample["DocumentId"][0]]

In [ ]:
docs2q

In [ ]:
sample['DocumentId']

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def doc_reader ():
    for idx, title, body in tqdm.tqdm (file_reader (), total=582167):
        yield dct.doc2bow(' '.join ([title,body]).split ())
            
corpora.MmCorpus.serialize('../result/{}/corpus.mm'.format (TRY_NAME), doc_reader ())

import gensim as gs
import math
import numpy as np
import tqdm

dct_title = gs.corpora.Dictionary.load ("../result/{}/dict.dct".format (TRY_NAME))

corpus = gs.corpora.MmCorpus('../result/{}/corpus.mm'.format (TRY_NAME))

PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

corp_size = 0
corp_size_words = 0
for doc in tqdm.tqdm (corpus):
    corp_size_words += sum (dict (doc).itervalues ())
    corp_size += 1

avgdl = float (corp_size_words) / corp_size

idfs = {}

summ_idf = 0

for word_id, freq in dct_title.dfs.iteritems ():
    idfs[word_id] = math.log(corp_size - freq + 0.5) - math.log(freq + 0.5)
    summ_idf += idfs[word_id]

average_idf = float (summ_idf) / len (dct_title.dfs)

for id in idfs.iterkeys ():
    idfs[id] = idfs[id] if idfs[id] >= 0 else EPSILON * average_idf

q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").lower ().split ("\t")
        q_ids.append (int (q[0]))
        queries.append (dict (dct_title.doc2bow (apply_to_str (q[1]))))

q_size = len (queries)

In [ ]:
result = np.zeros ((q_size, corp_size))

In [ ]:
for doc_i, doc in tqdm.tqdm (enumerate (corpus), total=582167):
    doc = dict (doc)
    doc_keys = set (doc)
    doc_len = sum (doc.itervalues ())
    for q_i in docs2q[doc_i]:
        score = 0
        for word in set (queries[q_i].keys ()) & doc_keys:
            idf = idfs[word]
            score += (idf * doc[word] * (PARAM_K1 + 1)
                      / (doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len / avgdl)))
        result[q_i, doc_i] = score

with open ("../result/result_{}.csv".format (TRY_NAME), "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for q_num, qid in tqdm.tqdm (enumerate (q_ids)):
        for doc in np.argsort (result[q_num, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

# np.save (open ("../result/{}/ranking.npy".format (TRY_NAME), "wb"), result)

In [ ]:
with open ("../result/result_{}.csv".format (TRY_NAME), "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for q_num, qid in tqdm.tqdm (enumerate (q_ids)):
        for doc in np.argsort (result[q_num, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

In [ ]:
# titiles stemmed

In [ ]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora
from gensim import corpora, models, similarities
import numpy as np
import logging

TRY_NAME = "WORDS_TITLE"
title = True

dct = Dictionary(prune_at=None)

with open ("../data/docs.tsv") as fin:
    doc_num = 1 if title else 2
    
    for doc in tqdm.tqdm (fin):
        doc = doc.decode ("utf-8").strip ().split("\t")
        if len (doc) > doc_num:
            dct.add_documents ([doc[doc_num].split ()], prune_at=None)

dct.save ("../result/{}/dict.dct".format (TRY_NAME))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def doc_reader (title):
    doc_num = 0 if title else 1
    
    with open ("../data/docs.tsv") as fin:
        for i, doc in enumerate (fin):
            doc = doc.decode ("utf-8").strip ().split("\t")[1:]
            if len (doc) > doc_num:
                yield dct.doc2bow(doc[doc_num].split ())
            else:
                yield []

corpora.MmCorpus.serialize('../result/{}/corpus.mm'.format (TRY_NAME), doc_reader (title))

import gensim as gs
import math
import numpy as np
import tqdm

dct_title = gs.corpora.Dictionary.load ("../result/{}/dict.dct".format (TRY_NAME))

corpus = gs.corpora.MmCorpus('../result/{}/corpus.mm'.format (TRY_NAME))

PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

corp_size = 0
corp_size_words = 0
for doc in tqdm.tqdm (corpus):
    corp_size_words += sum (dict (doc).itervalues ())
    corp_size += 1

avgdl = float (corp_size_words) / corp_size

idfs = {}

summ_idf = 0

for word_id, freq in dct_title.dfs.iteritems ():
    idfs[word_id] = math.log(corp_size - freq + 0.5) - math.log(freq + 0.5)
    summ_idf += idfs[word_id]

average_idf = float (summ_idf) / len (dct_title.dfs)

for id in idfs.iterkeys ():
    idfs[id] = idfs[id] if idfs[id] >= 0 else EPSILON * average_idf

q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").upper ().split ("\t")
        q_ids.append (int (q[0]))
        queries.append (dict (dct_title.doc2bow (q[1].split ())))

q_size = len (queries)

result = np.zeros ((q_size, corp_size))

for doc_i, doc in tqdm.tqdm (enumerate (corpus)):
    doc = dict (doc)
    doc_keys = set (doc)
    doc_len = sum (doc.itervalues ())
    for q_i, q in enumerate (queries):
        score = 0
        for word in set (q.keys ()) & doc_keys:
            idf = idfs[word]
            score += (idf * doc[word] * (PARAM_K1 + 1)
                      / (doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len / avgdl)))
        result[q_i, doc_i] = score

with open ("../result/result_{}.csv".format (TRY_NAME), "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for q_num, qid in tqdm.tqdm (enumerate (q_ids)):
        for doc in np.argsort (result[q_num, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

np.save (open ("../result/{}/ranking.npy".format (TRY_NAME), "wb"), result)


In [ ]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora, models, similarities
import numpy as np
import logging
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import pandas as pd

lemmatizer = Mystem()
stemmer = SnowballStemmer('russian', ignore_stopwords=True)
stop = stopwords.words('russian')
TRY_NAME = "TITLE_STEM"
stop.extend (["\n", " "])
files = ['xaa','xab','xac','xad','xae','xaf','xag','xah','xai','xaj','xak','xal']

In [ ]:
def file_reader ():
    for f in files:
        with open ("../temp/" + f + "_result") as fin:
            for l in fin:
                res = l.decode ("utf-8").lower ().strip ().split("\t")
                idx = res[0]
                title = ""
                body = ""
                if len (res) > 1:
                    title = res[1]
                if len (res) > 2:
                    body = res[2]
                yield idx, title, body

In [ ]:
dct = Dictionary(prune_at=None)

def apply_to_str (string):
    no_stops = filter (lambda x: x not in stop, lemmatizer.lemmatize (string))
    return map(lambda x: stemmer.stem(x), no_stops)

for idx, title, _ in tqdm.tqdm (file_reader (), total=582167):
    dct.add_documents ([title.split ()], prune_at=None)

In [ ]:
dct.save ("../result/{}/dict.dct".format (TRY_NAME))

In [ ]:
sample = pd.read_csv('../data/sample.csv', sep=',')

In [ ]:
docs2q = []
for i in tqdm.trange (583000):
    docs2q.append (list (sample[sample['DocumentId'] == i]['QueryId']))

In [ ]:
def doc_reader ():
    for idx, title, _ in tqdm.tqdm (file_reader (), total=582167):
        yield dct.doc2bow(title.split ())
            
corpora.MmCorpus.serialize('../result/{}/corpus.mm'.format (TRY_NAME), doc_reader ())

import gensim as gs
import math
import numpy as np
import tqdm

dct_title = gs.corpora.Dictionary.load ("../result/{}/dict.dct".format (TRY_NAME))

corpus = gs.corpora.MmCorpus('../result/{}/corpus.mm'.format (TRY_NAME))

PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

corp_size = 0
corp_size_words = 0
for doc in tqdm.tqdm (corpus):
    corp_size_words += sum (dict (doc).itervalues ())
    corp_size += 1

avgdl = float (corp_size_words) / corp_size

idfs = {}

summ_idf = 0

for word_id, freq in dct_title.dfs.iteritems ():
    idfs[word_id] = math.log(corp_size - freq + 0.5) - math.log(freq + 0.5)
    summ_idf += idfs[word_id]

average_idf = float (summ_idf) / len (dct_title.dfs)

for id in idfs.iterkeys ():
    idfs[id] = idfs[id] if idfs[id] >= 0 else EPSILON * average_idf

q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").lower ().split ("\t")
        q_ids.append (int (q[0]))
        queries.append (dict (dct_title.doc2bow (apply_to_str (q[1]))))

q_size = len (queries)

In [ ]:
result = np.zeros ((q_size, corp_size))

In [ ]:
for doc_i, doc in tqdm.tqdm (enumerate (corpus), total=582167):
    doc = dict (doc)
    doc_keys = set (doc)
    doc_len = sum (doc.itervalues ())
    for q_i in docs2q[doc_i]:
        score = 0
        for word in set (queries[q_i].keys ()) & doc_keys:
            idf = idfs[word]
            score += (idf * doc[word] * (PARAM_K1 + 1)
                      / (doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len / avgdl)))
        result[q_i, doc_i] = score

with open ("../result/result_{}.csv".format (TRY_NAME), "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for q_num, qid in tqdm.tqdm (enumerate (q_ids)):
        for doc in np.argsort (result[q_num, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

In [1]:
#title old style

In [ ]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora, models, similarities
import numpy as np
import logging
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

lemmatizer = Mystem()
stemmer = SnowballStemmer('russian', ignore_stopwords=True)
stop = stopwords.words('russian')
TRY_NAME = "STEM_TIT"
stop.extend (["\n", " "])

def apply_to_str (string):
    no_stops = filter (lambda x: x not in stop, lemmatizer.lemmatize (string))
    return map(lambda x: stemmer.stem(x), no_stops)

title = True

dct = Dictionary(prune_at=None)

with open ("../data/docs.tsv") as fin:
    doc_num = 1 if title else 2
    
    for doc in tqdm.tqdm (fin):
        doc = doc.decode ("utf-8").strip ().lower ().split("\t")
        if len (doc) > doc_num:
            dct.add_documents ([apply_to_str (doc[doc_num])], prune_at=None)

In [4]:
dct.save ("../result/{}/dict.dct".format (TRY_NAME))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def doc_reader (title):
    doc_num = 0 if title else 1
    
    with open ("../data/docs.tsv") as fin:
        for i, doc in enumerate (fin):
            doc = doc.decode ("utf-8").lower ().strip ().split("\t")[1:]
            if len (doc) > doc_num:
                yield dct.doc2bow(apply_to_str (doc[doc_num]))
            else:
                yield []

corpora.MmCorpus.serialize('../result/{}/corpus.mm'.format (TRY_NAME), doc_reader (title))

import gensim as gs
import math
import numpy as np
import tqdm

dct_title = gs.corpora.Dictionary.load ("../result/{}/dict.dct".format (TRY_NAME))

corpus = gs.corpora.MmCorpus('../result/{}/corpus.mm'.format (TRY_NAME))

PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

corp_size = 0
corp_size_words = 0
for doc in tqdm.tqdm (corpus):
    corp_size_words += sum (dict (doc).itervalues ())
    corp_size += 1

avgdl = float (corp_size_words) / corp_size

idfs = {}

summ_idf = 0

for word_id, freq in dct_title.dfs.iteritems ():
    idfs[word_id] = math.log(corp_size - freq + 0.5) - math.log(freq + 0.5)
    summ_idf += idfs[word_id]

average_idf = float (summ_idf) / len (dct_title.dfs)

for id in idfs.iterkeys ():
    idfs[id] = idfs[id] if idfs[id] >= 0 else EPSILON * average_idf

2018-08-24 13:10:56,495 : INFO : storing corpus in Matrix Market format to ../result/STEM_TIT/corpus.mm
2018-08-24 13:10:56,497 : INFO : saving sparse matrix to ../result/STEM_TIT/corpus.mm
2018-08-24 13:10:56,547 : INFO : PROGRESS: saving document #0
2018-08-24 13:11:01,641 : INFO : PROGRESS: saving document #1000
2018-08-24 13:11:03,549 : INFO : PROGRESS: saving document #2000
2018-08-24 13:11:05,000 : INFO : PROGRESS: saving document #3000
2018-08-24 13:11:07,028 : INFO : PROGRESS: saving document #4000
2018-08-24 13:11:08,726 : INFO : PROGRESS: saving document #5000
2018-08-24 13:11:11,541 : INFO : PROGRESS: saving document #6000
2018-08-24 13:11:15,947 : INFO : PROGRESS: saving document #7000
2018-08-24 13:11:17,475 : INFO : PROGRESS: saving document #8000
2018-08-24 13:11:19,301 : INFO : PROGRESS: saving document #9000
2018-08-24 13:11:21,511 : INFO : PROGRESS: saving document #10000
2018-08-24 13:11:24,385 : INFO : PROGRESS: saving document #11000
2018-08-24 13:11:26,149 : INFO 

2018-08-24 13:16:00,925 : INFO : PROGRESS: saving document #122000
2018-08-24 13:16:02,725 : INFO : PROGRESS: saving document #123000
2018-08-24 13:16:04,299 : INFO : PROGRESS: saving document #124000
2018-08-24 13:16:05,535 : INFO : PROGRESS: saving document #125000
2018-08-24 13:16:07,203 : INFO : PROGRESS: saving document #126000
2018-08-24 13:16:09,226 : INFO : PROGRESS: saving document #127000
2018-08-24 13:16:11,691 : INFO : PROGRESS: saving document #128000
2018-08-24 13:16:14,112 : INFO : PROGRESS: saving document #129000
2018-08-24 13:16:16,930 : INFO : PROGRESS: saving document #130000
2018-08-24 13:16:23,025 : INFO : PROGRESS: saving document #131000
2018-08-24 13:16:26,676 : INFO : PROGRESS: saving document #132000
2018-08-24 13:16:28,185 : INFO : PROGRESS: saving document #133000
2018-08-24 13:16:30,188 : INFO : PROGRESS: saving document #134000
2018-08-24 13:16:32,172 : INFO : PROGRESS: saving document #135000
2018-08-24 13:16:34,321 : INFO : PROGRESS: saving document #13

2018-08-24 13:20:28,511 : INFO : PROGRESS: saving document #245000
2018-08-24 13:20:30,217 : INFO : PROGRESS: saving document #246000
2018-08-24 13:20:32,914 : INFO : PROGRESS: saving document #247000
2018-08-24 13:20:35,544 : INFO : PROGRESS: saving document #248000
2018-08-24 13:20:37,204 : INFO : PROGRESS: saving document #249000
2018-08-24 13:20:38,866 : INFO : PROGRESS: saving document #250000
2018-08-24 13:20:40,246 : INFO : PROGRESS: saving document #251000
2018-08-24 13:20:41,711 : INFO : PROGRESS: saving document #252000
2018-08-24 13:20:44,027 : INFO : PROGRESS: saving document #253000
2018-08-24 13:20:45,597 : INFO : PROGRESS: saving document #254000
2018-08-24 13:20:47,286 : INFO : PROGRESS: saving document #255000
2018-08-24 13:20:49,288 : INFO : PROGRESS: saving document #256000
2018-08-24 13:20:50,826 : INFO : PROGRESS: saving document #257000
2018-08-24 13:20:53,382 : INFO : PROGRESS: saving document #258000
2018-08-24 13:20:55,385 : INFO : PROGRESS: saving document #25

2018-08-24 13:24:20,923 : INFO : PROGRESS: saving document #368000
2018-08-24 13:24:23,095 : INFO : PROGRESS: saving document #369000
2018-08-24 13:24:25,205 : INFO : PROGRESS: saving document #370000
2018-08-24 13:24:26,975 : INFO : PROGRESS: saving document #371000
2018-08-24 13:24:28,767 : INFO : PROGRESS: saving document #372000
2018-08-24 13:24:30,851 : INFO : PROGRESS: saving document #373000
2018-08-24 13:24:38,114 : INFO : PROGRESS: saving document #374000
2018-08-24 13:24:40,364 : INFO : PROGRESS: saving document #375000
2018-08-24 13:24:42,087 : INFO : PROGRESS: saving document #376000
2018-08-24 13:24:43,905 : INFO : PROGRESS: saving document #377000
2018-08-24 13:24:45,483 : INFO : PROGRESS: saving document #378000
2018-08-24 13:24:47,830 : INFO : PROGRESS: saving document #379000
2018-08-24 13:24:50,275 : INFO : PROGRESS: saving document #380000
2018-08-24 13:24:53,021 : INFO : PROGRESS: saving document #381000
2018-08-24 13:24:54,677 : INFO : PROGRESS: saving document #38

2018-08-24 13:28:51,247 : INFO : PROGRESS: saving document #491000
2018-08-24 13:28:53,750 : INFO : PROGRESS: saving document #492000
2018-08-24 13:28:56,100 : INFO : PROGRESS: saving document #493000
2018-08-24 13:28:57,622 : INFO : PROGRESS: saving document #494000
2018-08-24 13:29:00,288 : INFO : PROGRESS: saving document #495000
2018-08-24 13:29:02,873 : INFO : PROGRESS: saving document #496000
2018-08-24 13:29:04,999 : INFO : PROGRESS: saving document #497000
2018-08-24 13:29:07,521 : INFO : PROGRESS: saving document #498000
2018-08-24 13:29:09,090 : INFO : PROGRESS: saving document #499000
2018-08-24 13:29:10,489 : INFO : PROGRESS: saving document #500000
2018-08-24 13:29:12,509 : INFO : PROGRESS: saving document #501000
2018-08-24 13:29:13,766 : INFO : PROGRESS: saving document #502000
2018-08-24 13:29:16,269 : INFO : PROGRESS: saving document #503000
2018-08-24 13:29:18,516 : INFO : PROGRESS: saving document #504000
2018-08-24 13:29:20,692 : INFO : PROGRESS: saving document #50

In [ ]:
q_ids = {}
queries = []
with open ("../data/queries.tsv") as fin:
    for i, q in enumerate (fin):
        q = q.strip ().decode ("utf-8").lower ().split ("\t")
        q_ids[int (q[0])] = i
        queries.append (dict (dct_title.doc2bow (apply_to_str (q[1]))))

q_size = len (queries)

result = np.zeros ((q_size, corp_size))

In [7]:
import pandas as pd

In [ ]:
sample = pd.read_csv('../data/sample.csv', sep=',')
docs2q = []
for i in tqdm.trange (583000):
    docs2q.append (list (sample[sample['DocumentId'] == i]['QueryId']))

for doc_i, doc in tqdm.tqdm (enumerate (corpus)):
    doc = dict (doc)
    doc_keys = set (doc)
    doc_len = sum (doc.itervalues ())
    for q_i in docs2q[doc_i]:
        score = 0
        for word in set (queries[q_ids[q_i]].keys ()) & doc_keys:
            idf = idfs[word]
            score += (idf * doc[word] * (PARAM_K1 + 1)
                      / (doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len / avgdl)))
        result[q_i, doc_i] = score

In [11]:
with open ("../result/result_{}.csv".format (TRY_NAME), "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for qid, qnum in tqdm.tqdm (q_ids.iteritems ()):
        for doc in np.argsort (result[qnum, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

# np.save (open ("../result/{}/ranking.npy".format (TRY_NAME), "wb"), result)

6311it [01:05, 96.99it/s]


In [12]:
head "../result/{}/ranking.npy".format (TRY_NAME)

SyntaxError: invalid syntax (<ipython-input-12-02aefb79758a>, line 1)